# Dynex的量子体积

量子体积（Quantum Volume, QV）是用于衡量量子计算机性能的一个综合指标。它不仅仅考虑量子比特（qubits）的数量，还包括其他影响量子计算机实际计算能力的多个因素，如：

量子比特的数量：量子体积会考虑量子计算机中的有效量子比特数。
门的保真度：量子门操作的精确度，即在计算中执行逻辑门操作时的误差率。
连通性：量子比特之间的连通性，即量子比特如何相互作用。
并行性：同时操作多个量子比特的能力。
纠错能力：量子计算中的误差校正和纠错的效率。
量子体积越大，意味着量子计算机在处理复杂问题时表现得更好。它反映了量子计算机解决实际问题的能力，而不仅仅是单纯增加量子比特数量。

IBM提出了“量子体积”这一概念，作为评估不同量子计算机之间性能的通用标准。

参考文献: https://en.wikipedia.org/wiki/Quantum_volume

## 电路深度和宽度

量子体积测量量子计算机可以成功实现的最大随机电路。对于电路来说，宽度是量子位的数量 
𝑛，深度是应用的门的层数，也设置为𝑛以确保量子比特数和一致性的平衡评估。

QV 电路的深度为 𝑑=𝑛，这意味着它在量子位对之间应用随机选择的双量子位门的 𝑛 层。系统中的量子位和门越多，保持相干性和实现高精度就越困难，这就是为什么这种平衡方法可以深入了解量子系统的整体性能。

## 随机酉运算

在每个深度，应用随机酉矩阵。这些酉矩阵是从哈尔测度中选择的，这保证了运算在可能的量子运算中均匀分布。电路的每一层都由应用于随机选择的量子位对的随机单一门组成。

## 重输出概率

运行电路后，测量系统的输出概率。下一个关键要素是计算重输出概率（HOP），它将量子计算机的概率分布与理想分布（从酉矩阵导出的理论分布）进行比较。

重输出是指概率位于所有测量结果前 50% 的结果。
重输出概率 𝑃heavy 定义为在实验结果中观察到重输出的次数的分数。
从数学上讲，如果输出 𝑥 的实验概率分布为 𝑝(𝑥)，则重输出集 𝐻 包含具有最大 𝑝(𝑥) 值的输出，使得 𝐻 至少包含总概率质量的 50%。

## 目标

要使量子计算机成功实现电路，𝑃heavy 必须大于 2/3。该阈值表明量子系统的性能明显优于随机猜测

该指标捕获量子系统的大小及其保持相干性和执行有用量子计算的能力，使其成为量子计算性能的综合衡量标准。

In [15]:
import pennylane as qml
from pennylane import numpy as np
from scipy.stats import unitary_group
import numpy as np
import dynex

In [42]:
# 量子位数量
n_wires = 100 # QV = 2^n_wires, 记录是20

In [43]:
depth = [n_wires]
n_circuits = 1  # 要运行的随机电路数
threshold = 2 / 3

# 生成随机酉矩阵
def random_unitary_matrix():
    return unitary_group.rvs(2)

# 随机单一层
def random_layer(wires):
    for i in range(len(wires)):
        qml.QubitUnitary(random_unitary_matrix(), wires=wires[i])

# 量子体积电路
def quantum_volume_circuit(depth):
    for layer in range(depth[0]):
        random_layer(range(n_wires))
    return qml.state()

In [48]:
# 绘制电路：（对于较大的电路，PL 需要很长时间）
_ = qml.draw_mpl(quantum_volume_circuit, style="black_white")(depth)

In [44]:
# 运行多个随机电路并收集结果
heavy_outputs = 0

for _ in range(n_circuits):
    result = dynex.execute(quantum_volume_circuit, depth, n_wires, mainnet=True, is_cluster=True, method='probs', shots=10)
    sorted_probs = np.sort(result)
    print(sorted_probs)
    median_prob = np.median(sorted_probs)
    print('median_prob       :',median_prob)
    heavy_output_prob = np.sum(result[result > median_prob])
    print('heavy_output_prob :',heavy_output_prob)
    
    if heavy_output_prob > threshold:
        heavy_outputs += 1

# 计算成功率
success_rate = heavy_outputs / n_circuits

# 确定量子体积
if success_rate > threshold:
    quantum_volume = 2 ** n_wires
else:
    quantum_volume = None

print(f"Quantum Volume: {quantum_volume} (Success Rate: {success_rate * 100:.2f}%)")

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      45684 │      408 │            1464 │        0.00 │      0.80 │             10 │      10000 │     256 │        4830.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────┬───────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │   ENERGY │       RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────┼───────────────┼─────────────────